### MATH 350 Numerical Analysis Homework

In [635]:
import os
import time
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from typing import List, Union
from collections.abc import Callable
from decimal import Decimal, getcontext
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

In [636]:
pd.set_option('display.float_format', '{:.4f}'.format)

#### Brief Background

We denote $Y(x)$ as the true solution to the Initial Value Problem (IVP). The initial value is $Y_0$

\begin{cases} 
  Y'(x) = f(x, Y(x)), & \quad x_0 \leq x \leq b, \\
  Y(x_0) = Y_0 &
\end{cases}


\begin{align*}
x_n &= x_0 + nh, \quad n = 0, 1, \dots, N
\end{align*}

\begin{align*}
y(x_n) = y_h(x_n) = y_n \quad n = 0, 1, \dots, N
\end{align*}


#### Presented Problem

$$Y(x) = \tan^{-1}(x)$$ $$Y(0) = 0$$

#### Previous methods used

```python
# Problem 8.2-1a
def y(x: Union[int, float]) -> Union[np.float64, np.ndarray]:
    return np.arctan(x)
    
def y_p(x: Union[int, float]) -> Union[np.float64, np.ndarray]:
    return np.pow(np.cos(y(x)), 2)
    
# Problem 8.2-1b
def y(x: Union[int, float]) -> Union[np.float64, np.ndarray]:
    return np.divide(x, (1+np.pow(x, 2)))

def y_p(x: Union[int, float]) -> Union[np.float64, np.ndarray]:
    return np.divide(1, (1+np.pow(x, 2))) - 2*(np.pow(y(x), 2))
```

#### Actual Playground

In [637]:
def y_exact(x: Union[int, float]) -> Union[np.float64, np.ndarray]:
    return np.exp(-1 * x)

In [638]:
def y_prime(x: Union[int, float]) -> Union[np.float64, np.ndarray]:
    return -1 * y(x)

In [639]:
def eulers_method(l:int, u:int, h: Union[float, int], x0: Union[int, float], y0: Union[int, float],
                  y: Callable[[int, float], [np.float64, np.ndarray]],
                  yp: Callable[[int, float], [np.float64, np.ndarray]]
                 ):
    y_values = [y0]
    x_values = np.linspace(l, u, int((u-l)/h) + 1)

    for i, x_n in enumerate(x_values):
        y_n = y_values[-1]

        y_next = y_n + (h * yp(x_n))

        y_values.append(y_next)
        #print(f"Step {i}: x={x_n:.4f}, y_{i}={y_n:.4f} -> y_{i+1}={y_next:.4f}")
        #print("-" * 50)
        
    return x_values ,y_values[1:]

In [640]:
def bound(b:int, h:int) -> float:
    return np.divide(h, 2) * (np.exp(b) - 1)

In [641]:
initial_x = 0
initial_y = 1
lower_bound = 0
upper_bound = [1, 2, 3, 4, 5]
h_values = [0.2, 0.1, 0.05]

In [642]:
x_outputs, y_outputs = eulers_method(l=lower_bound, u=upper_bound[1], h=h_values[1],
                                      x0=initial_x, y0=initial_y,
                                      y=y_exact, yp=y_prime)

In [643]:
h_values = np.full(len(x_outputs), h_values[1])
true_values = [y(x) for x in x_outputs]
bound_values = np.full(len(x_outputs), bound(upper_bound[1], h_values[1]))
errors = [(true_values[i] - y) for i, y in enumerate(y_outputs)]
errors_bool = [errors[i] < bound for i, bound in enumerate(bound_values)]

In [644]:
titles = [
    r"$h_{n}$",
    r"$x_{n}$",
    r"$y_{h}$",
    r"$e^{-x_n}$",
    r"$\frac{h}{2}(e^5-1)$", 
    r"$e^{-x_n} - y_h(x_n)$",
    r"$e^{-x_n} - y_h(x_n) \leq \frac{h}{2}(e^5 - 1)$",
]

In [645]:
data = pd.DataFrame([
    h_values,
    x_outputs,
    y_outputs,
    true_values,
    bound_values,
    errors,
    errors_bool,
]).T
data.columns = titles

In [646]:
plt.figure(figsize=(6, 5))
plt.plot(data[r"$x_{n}$"], data[r"$y_{h}$"], label="Euler's Method Output")
plt.plot(data[r"$x_{n}$"], data[r"$e^{-x_n}$"], label="Exact Solution", linestyle='--')
plt.title(r'Exact Solution vs Euler\'s Method Approximation', fontsize=14)
plt.xlabel(r'$x_{n}$', fontsize=12)
plt.ylabel(r'$y(x_{n})$', fontsize=12)
plt.grid(True)
plt.legend()
plt.close()

The code below will generate `MD` tables of the data and the graph to go with it 

```python
save_folder = "./lab_data/"
os.makedirs(save_folder, exist_ok=True)

initial_x = 0
initial_y = 1
lower_bound = 0
upper_bound = [1, 2, 3, 4, 5]
h_possibles = [0.2, 0.1, 0.05]

for ub in upper_bound:
    for hv in h_possibles:
        x_outputs, y_outputs = eulers_method(
            l=lower_bound, u=ub, h=hv,
            x0=initial_x, y0=initial_y,
            y=y_exact, yp=y_prime
        )
        h_values = np.full(len(x_outputs), hv)
        true_values = [y(x) for x in x_outputs]
        bound_values = np.full(len(x_outputs), bound(ub, hv))
        errors = [(true_values[i] - y) for i, y in enumerate(y_outputs)]
        errors_bool = [errors[i] < bound for i, bound in enumerate(bound_values)]
        titles = [
            r"$h_{n}$",
            r"$x_{n}$",
            r"$y_{h}$",
            r"$e^{-x_n}$",
            r"$\frac{h}{2}(e^5-1)$", 
            r"$e^{-x_n} - y_h(x_n)$",
            r"$e^{-x_n} - y_h(x_n) \leq \frac{h}{2}(e^5 - 1)$",
        ]
        
        data = pd.DataFrame([
            h_values,
            x_outputs,
            y_outputs,
            true_values,
            bound_values,
            errors,
            errors_bool,
        ]).T
        
        data.columns = titles
        data.index = np.full(len(x_outputs), ub)
        data.to_markdown(f"./lab_data/euler_table_b_{ub}_h_{hv}.md")        

        plot_filename = f"./lab_data/euler_table_b_{ub}_h_{hv}.png"
        
        plt.figure(figsize=(12, 5))
        plt.plot(data[r"$x_{n}$"], data[r"$y_{h}$"], label="Euler's Method Output")
        plt.plot(data[r"$x_{n}$"], data[r"$e^{-x_n}$"], label="Exact Solution", linestyle='--')
        plt.title(r'Exact Solution vs Euler\'s Method Approximation', fontsize=14)
        plt.xlabel(r'$x_{n}$', fontsize=12)
        plt.ylabel(r'$y(x_{n})$', fontsize=12)
        plt.grid(True)
        plt.legend()
        plt.savefig(plot_filename)  # Save the plot as PNG
        plt.close() 

        time.sleep(4)
```

#### Problem 8.3 - 9

Recall that Richardson's extrapolation formula is:
$$
Y(x) \approx y_h(x) \approx 2y_h(x) - y_{2h}(x)
$$

To find the error, we'll perform:

$$
\left[ 2y_h(x) - y_{2h}(x) \right] - y_h(x_n)
$$

The conditions are:

$$
Y^{'}(x) = \frac{Y(x) + x^2 - 2}{x+1},\,\, Y(x) = x^2 + 2x + 2 - 2(x+1)\log(x+1)
$$
$$
Y(0) = 2
$$

In [647]:
def y_exact(x: Union[int, float]) -> Union[np.float64, np.ndarray]:
    return np.pow(x, 2)+(2*x)+2-(2*(x+1)*np.log(x+1))

def y_prime(x: Union[int, float]) -> Union[np.float64, np.ndarray]:
    return np.divide((y_exact(x)+np.pow(x, 2)-2),(x+1))

Therefore, let 2$y_h(x)$ be:

In [623]:
richard_x_1, richard_y_1 = eulers_method(l=1, u=6, h=0.05,
                                          x0=0, y0=2,
                                          y=y_exact, yp=y_prime)

In [624]:
richard_y_1 = np.array([(2*y) for y in richard_y_1])

And let $y_{2h}(x)$ be:

In [625]:
richard_x_2, richard_y_2 = eulers_method(l=1, u=6, h=(2*0.05),
                                          x0=0, y0=2,
                                          y=y_exact, yp=y_prime)

In [626]:
richard_y_2 = np.array(richard_y_2)

In [627]:
richard_x_subtract, richard_y_subtract = eulers_method(l=1, u=6, h=0.05,
                                          x0=0, y0=2,
                                          y=y_exact, yp=y_prime)

In [628]:
richard_y_subtract = np.array(richard_y_subtract)

In [629]:
mask = np.nonzero(np.isin(richard_x_1, richard_x_2))[0]

In [630]:
richard_x_1 = richard_x_1[mask]
richard_y_1 = richard_y_1[mask]
richard_x_subtract = richard_x_subtract[mask]
richard_y_subtract = richard_y_subtract[mask]

In [631]:
richardsons = np.abs(richard_y_2 - richard_y_1)
richardsons_extrapolate = np.abs(richard_y_subtract - richard_y_2)

In [632]:
titles = [
    r"$h_{n}$",
    r"$2h_{n}$",
    r"$x_{n}$",
    r"$2y_h(x)$",
    r"$y_{2h}(x)$",
    r"$y_h(x)$",
    r"Richardson's",
    r"$y_h(x) - y_{2h}(x)$",
]

In [633]:
data = pd.DataFrame([
    np.full(richard_x_1.shape[0], 0.05),
    np.full(richard_x_1.shape[0], (2*0.5)),
    richard_x_1,
    richard_y_1,
    richard_y_2,
    richard_y_subtract,
    richardsons,
    richardsons_extrapolate,
]).T
data.columns = titles

In [649]:
def y_exact(x: Union[int, float]) -> Union[np.float64, np.ndarray]:
    return np.sin(x) + np.cos(X)

def y_prime(x: Union[int, float], l) -> Union[np.float64, np.ndarray]:
    return l * y_exact(x) + (1-l)*np.cos(x) - (1+l)*np.sin(x)

In [ ]:
def eulers_method(l:int, u:int, h: Union[float, int], x0: Union[int, float], y0: Union[int, float],
                  y: Callable[[int, float], [np.float64, np.ndarray]],
                  yp: Callable[[int, float], [np.float64, np.ndarray]]
                 ):
    y_values = [y0]
    x_values = np.linspace(l, u, int((u-l)/h) + 1)

    for i, x_n in enumerate(x_values):
        y_n = y_values[-1]

        y_next = y_n + (h * yp(x_n))

        y_values.append(y_next)
        #print(f"Step {i}: x={x_n:.4f}, y_{i}={y_n:.4f} -> y_{i+1}={y_next:.4f}")
        #print("-" * 50)
        
    return x_values ,y_values[1:]